In [ ]:
import src.controller as controller
import src.bb.blackboard_optimization as bb_opt
from src.ka.ka_s.pymoo_plugin import PyMooAlgorithm
from src.ka.ka_brs.level3 import KaBrLevel3
from src.ka.ka_brs.level2 import KaBrLevel2
from src.ka.ka_brs.level1 import KaBrLevel1
from src.ka.ka_s.stochastic import Stochastic
from src.ka.ka_s.latin_hypercube import LatinHypercube
from src.ka.ka_s.neighborhood_search import NeighborhoodSearch
from src.ka.ka_s.hill_climb import HillClimb
from src.ka.ka_s.genetic_algorithm import GeneticAlgorithm
from src.utils.problem import BenchmarkProblem

import shutil
import time
import plotly.graph_objects as go
from osbrain import run_nameserver
from osbrain import run_agent
import plotly.express as px
import os
import numpy as np
import Pyro4
Pyro4.config.THREADPOOL_SIZE = 300
import numpy.random as random
import copy

In [ ]:
def BenchmarkController(bb, kas, problem, num, rand=None):

    bb_controller = controller.Controller()
    bb_controller.initialize_blackboard(blackboard=bb,
                                  ka=kas, 
                                  agent_wait_time=10,
                                  plot_progress=False,
                                  random_seed=rand,
                                  problem=problem,
                                       ) 
    
    bb = bb_controller.bb_opt
    server = bb.get_attr('_proxy_server')
    for agent in kas.keys():
        agent_ = server.proxy(agent)
        agent_.set_attr(_design_accuracy=18)
        agent_.set_attr(_objective_accuracy=18)
        agent_.set_attr(debug_wait=True)
        agent_.set_attr(debug_wait_time=10)
        if 'pymoo' in agent:
            agent_.set_attr(debug_wait=True)            
            #agent_.set_attr(debug_wait_time=0.25)
   
    bb_controller.run_multi_agent_bb('bb_opt')    
    print(bb.get_blackboard()['level 100']['final'])
    print()
    time.sleep(30.0)
    bb_controller.shutdown()
    time.sleep(0.05)
    return bb

def run_algorithm(dv, obj, const, model):
    problem = BenchmarkProblem(design_variables=dvs,
                         objectives=objs,
                         constraints=const,
                         benchmark_name = model) 
    dvi_ = {obj_name: 2.5 for obj_name, obj_dict in obj.items()}


    bb = {'name': 'bb_opt', 'type': bb_opt.BbOpt, 'attr': {'archive_name': '{}.h5'.format(model), 
                                                       'convergence_interval': 25,
                                                       'convergence_type': 'function evals',
                                                       'function_evals': 5_000,
                                                       'pf_size': 500,
                                                       'meta_data_to_log': ['function evals', 'hvi', 'gd', 'igd']}}
#    bb = {'name': 'bb_opt', 'type': bb_opt.BbOpt, 'attr': {'archive_name': '{}.h5'.format(model), 
#                                                       'convergence_interval': 25,
#                                                       'convergence_type': 'trigger evals',
#                                                       'total_tvs': 1_500,
#                                                       'pf_size': 500,
#                                                       'meta_data_to_log': ['function evals', 'hvi', 'gd', 'igd']}}

    kas = {'ka_rp_explore': {'type': Stochastic, 'attr' : {}}, 
       'ka_rp_lhc': {'type': LatinHypercube, 'attr' : {'samples': len(dvs)*100,'debug_wait':False, 'execute_once': True}},
       'ka_rp_lhc2': {'type': LatinHypercube, 'attr' : {'samples': len(dvs)*100,'debug_wait':False, 'lhc_criterion': 'simple', 'execute_once': True}},
       'ka_rp_hc1': {'type': HillClimb, 'attr' : {'step_size':0.01,'step_limit':50, 'reanalyze_designs': True}},
       'ka_rp_hc2': {'type': HillClimb, 'attr' : {'hc_type':'steepest ascent','step_limit':100}},
       'ka_rp_pert1': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.01, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert4': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.01, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert5': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.01, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert6': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.01, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert2': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.005}},
       'ka_rp_pert3': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.005, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pymoo2': {'type': PyMooAlgorithm, 'attr' : {'pymoo_algorithm_name': 'nsga2', 'pop_size': 50, 'n_offspring': 50, 'termination_criteria':2500, 'execute_once': True}},
       'ka_br_lvl3': {'type': KaBrLevel3, 'attr' : {}},
       'ka_br_lvl2': {'type': KaBrLevel2, 'attr' : {}},
       'ka_br_lvl1': {'type': KaBrLevel1, 'attr' : {}}}#'total_pf_size':500, 'pareto_sorter':'dci', 'dci_div':dvi_}}}


    for num in range(10,20):
        BenchmarkController(bb, kas, problem, num, rand=None)
        directory = model
        if not os.path.isdir('benchmarks/{}'.format(directory)):
            os.mkdir('benchmarks/{}'.format(directory))
        shutil.move('{}.h5'.format(model, num), 'benchmarks/{}/{}_{}.h5'.format(directory,model,num))
        time.sleep(5) 

# Parallelization Study

In [ ]:
def BenchmarkController(bb, kas, problem, num, base_time, rand=None):

    bb_controller = controller.Controller()
    bb_controller.initialize_blackboard(blackboard=bb,
                                  ka=kas, 
                                  agent_wait_time=300,
                                  plot_progress=False,
                                  random_seed=rand,
                                  problem=problem,
                                       ) 
    
    bb = bb_controller.bb_opt
    server = bb.get_attr('_proxy_server')
    for agent in kas.keys():
        agent_ = server.proxy(agent)
        agent_.set_attr(_design_accuracy=18)
        agent_.set_attr(_objective_accuracy=18)
        agent_.set_attr(debug_wait=True)
        agent_.set_attr(debug_wait_time=base_time)        
        if 'pymoo' in agent:
            agent_.set_attr(debug_wait_time=base_time/agent_.get_attr('pop_size'))
#        elif 'lhc' in agent:
#            agent_.set_attr(debug_wait_time=base_time/agent_.get_attr('samples'))   
#        elif 'ga' in agent:
#            agent_.set_attr(debug_wait_time=base_time/agent_.get_attr('offspring_per_generation'))      
#        elif 'ns' in agent:
#            agent_.set_attr(debug_wait_time=base_time/(len(agent_.get_attr('_design_variables'))*2)                  )
        else:
            agent_.set_attr(debug_wait_time=base_time)
                    
   
    bb_controller.run_multi_agent_bb('bb_opt')    
    #bb_controller.run_single_agent_bb('bb_opt')    
    data_meta = bb.get_blackboard()['level 100']['final']
    time.sleep(30.0)
    bb_controller.shutdown()
    time.sleep(0.05)
    return data_meta

def run_parallel_algorithm(dv, obj, const, model, num_agent_list, base_time):
    problem = BenchmarkProblem(design_variables=dvs,
                         objectives=objs,
                         constraints=const,
                         benchmark_name = model) 
    dvi_ = {obj_name: 2.5 for obj_name, obj_dict in obj.items()}


    bb = {'name': 'bb_opt', 'type': bb_opt.BbOpt, 'attr': {'archive_name': '{}_{}A_{}s.h5'.format(model,len(num_agent_list),base_time), 
                                                       'convergence_interval': 10,
                                                       'convergence_type': 'function evals',
                                                       'function_evals': 5_000,
                                                       'pf_size': 500,
                                                       'skipped_tvs': 10,
                                                       'meta_data_to_log': ['function evals', 'hvi', 'gd', 'igd'],
                                                       'write_h5': False}}

    stochastic_base = {'type': Stochastic, 'attr' : {}}
    stochastic_options = {}
    lhc_base = {'type': LatinHypercube, 'attr' : {'samples': len(dvs)*100, 'lhc_criterion': 'simple', 'execute_once': False}}
    lhc_options ={'lhc_criterion': ['simple', 'corr']}
    hc_base = {'type': HillClimb, 'attr' : {'hc_type': 'simple', 'converge criteria': 0.001, 'step_size':0.1, 'step_rate': 0.1, 'step_limit':100, 'reanalyze_designs': True}}
    hc_options = {'hc_type': ['simple', 'steepest ascent'], 'converge criteria': (0.0001,0.1), 'step_size': (0.01,0.5), 'step_rate': (0.01,0.5), 'step_limit':(5,10), 'reanalyze_designs': [True,False]}
    ns_base = {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.05, 'neighboorhood_search':'fixed','reanalyze_designs': True}}
    ns_options = {'perturbation_size':(0.005,0.5), 'neighboorhood_search': ['fixed', 'random'], 'reanalyze_designs': [True,False]}
    ga_base = {'type': GeneticAlgorithm, 'attr': {'offspring_per_generation':20, 'mutation_rate':0.1, 'perturbation_size':0.05,
                                                  'crossover_type': 'single point', 'mutation_type':'random', 'k':5, 'T':100, 'reanalyze_designs': True}}
    ga_options = {'crossover_type': ['linear crossover', 'single point'], 'mutation_type': ['random','non-uniform'], 'offspring_per_generation': (10,100),
                  'mutation_rate':(0.05, 0.25), 'k': (1,10), 'T':(50,150), 'reanalyze_designs': [True,False]}
    pymoo_base = {'type': PyMooAlgorithm, 'attr' : {'pymoo_algorithm_name': 'nsga2', 'pop_size': 50, 'n_offspring': 50, 'termination_criteria':2500, 'execute_once': True}}
    pymoo_options = {'pop_size': (25,150)}
    
    ka_options = {'mc': {'base': stochastic_base, 'options': stochastic_options},
                  'lhc': {'base': lhc_base, 'options': lhc_options},
                  'h-c': {'base': hc_base, 'options': hc_options},
                  'ns': {'base': ns_base, 'options': ns_options},
                  'ga': {'base': ga_base, 'options': ga_options},
                  'pymoo': {'base': pymoo_base, 'options': pymoo_options},}
                  
    kas = {'ka_br_lvl3': {'type': KaBrLevel3, 'attr' : {}},
           'ka_br_lvl2': {'type': KaBrLevel2, 'attr' : {}},
           'ka_br_lvl1': {'type': KaBrLevel1, 'attr' : {}}}
    
    random.seed(101)
    pop_size = 20
    for num_agents in num_agent_list:
        for ka, num_agent in num_agents.items():
            old_kas = copy.copy(kas)
            for num in range(num_agent):
                prev_num = num + sum([1 if ka in x else 0 for x in old_kas.keys()])
                ka_name = 'ka_s_{}_{}'.format(ka, prev_num)
                kas[ka_name] = copy.deepcopy(ka_options[ka]['base'])
                for opt, val in ka_options[ka]['options'].items():
                    if 'pymoo' in ka:
                        kas[ka_name]['attr'] = {'pop_size': pop_size, 'n_offspring': random.randint(low=pop_size/2,high=pop_size), 'termination_criteria': 1000, 'execute_once': random.choice([True, False])}
                        pop_size += 20
                    elif type(val) == list:
                        kas[ka_name]['attr'][opt] = random.choice(val)
                    else:
                        kas[ka_name]['attr'][opt] = random.uniform(low=val[0],high=val[1])

                    
#    for ka, ka_dict in kas.items():
#        print(ka, ka_dict)
#        print('')
    bb_list = []
    for num in range(0,10):
        time1 = time.time()
        meta_data = BenchmarkController(bb, kas, problem, num, base_time, rand=None)
        bb_list.append(meta_data)
#        print('*********************************')   
#        print(time.time()-time1)
#        directory = model
#        if not os.path.isdir('benchmarks/parallel/{}'.format(directory)):
#            os.mkdir('benchmarks/parallel/{}'.format(directory))
#        shutil.move('{}.h5'.format(model, num), 'benchmarks/parallel/{}/{}_{}.h5'.format(directory,model,num))
        time.sleep(5) 
    return bb_list

In [ ]:
model = 're32'
base_time = 3

objs = {'f0': {'ll':0.0,      'ul':35.5,        'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,      'ul':8_500_000.0,  'goal':'lt', 'variable type': float},
        'f2': {'ll':0.0,      'ul':426_000_000.0, 'goal':'lt', 'variable type': float},}
dvs =  {'x0':  {'ll': 0.125,  'ul':5.0,  'variable type': float},
        'x1':  {'ll': 0.1,    'ul':10.0, 'variable type': float},
        'x2':  {'ll': 0.1,    'ul':10.0, 'variable type': float},
        'x3':  {'ll': 0.125,  'ul':5.0, 'variable type': float}}
const = {}
num_agents = [{'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              {'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              {'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              {'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              {'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              {'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              {'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              {'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},]
bb_list = run_parallel_algorithm(dvs, objs, const, model, num_agents, base_time)
print(bb_list)

In [ ]:
meta = [{'function evals': 5306.0, 'hvi': 0.9987159870286868, 'gd': 13545.62372128425, 'igd': 14871369.861052953, 'agent': 'final', 'time': 173.30706787109375}, {'function evals': 5249.0, 'hvi': 0.9990509103350764, 'gd': 7878.4410239753115, 'igd': 25073737.30526427, 'agent': 'final', 'time': 143.4419708251953}, {'function evals': 5308.0, 'hvi': 0.9987372236063421, 'gd': 6149.899972338535, 'igd': 27207893.056309972, 'agent': 'final', 'time': 158.89577388763428}, {'function evals': 5154.0, 'hvi': 0.9991441661116729, 'gd': 8187.984167859975, 'igd': 41980911.41701728, 'agent': 'final', 'time': 187.5201930999756}, {'function evals': 5022.0, 'hvi': 0.9988795249952862, 'gd': 14233.629952707583, 'igd': 25129583.580892578, 'agent': 'final', 'time': 166.47065424919128}]

hvi = [result['hvi'] for result in meta]
gd = [result['gd'] for result in meta]
igd= [result['igd'] for result in meta]
time = [result['time'] for result in meta]
import math
def mean_std(results):
    mean = sum([x for x in results]) / len(results)
    var  = sum(pow(x-mean,2) for x in results) / len(results)
    std  = math.sqrt(var)
    return mean, std
        
def print_results(gd, gdp, hv, pf_len):
    m,s = mean_std(gd)
    print('Mean: {:.3e} ({:.3e})   Max: {}   Min  {}'.format(m,s,max(gd),min(gd)))
    m,s = mean_std(gdp)
    print('Mean: {:.3e} ({:.3e})   Max: {}   Min  {}'.format(m,s,max(gdp),min(gdp)))
    m,s = mean_std(hv)
    print('Mean: {:.3e} ({:.3e})   Max: {}   Min  {}'.format(m,s,max(hv),min(hv)))
    m,s = mean_std(pf_len)
    print('Mean: {:.3e} ({:.3e})   Max: {}   Min  {}'.format(m,s,max(pf_len),min(pf_len)))  
    
print_results(gd, igd, hvi, time)

In [ ]:
model = 're61'
base_time = 0.025

objs = {'f0': {'ll':63_000.0, 'ul':77_000.0, 'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,  'ul':1_400.0, 'goal':'lt', 'variable type': float},   
        'f2': {'ll':250_000.0,  'ul':2_900_000.0, 'goal':'lt', 'variable type': float},
        'f3': {'ll':70_000.0, 'ul':16_000_000.0, 'goal':'lt', 'variable type': float},
        'f4': {'ll':0.0,  'ul':350_000.0, 'goal':'lt', 'variable type': float},
        'f5': {'ll':0.0,  'ul':95_000.0, 'goal':'lt', 'variable type': float}}
dvs =  {'x0':  {'ll': 0.01,  'ul':0.45,  'variable type': float},
        'x1':  {'ll': 0.01,  'ul':0.1,  'variable type': float},
        'x2':  {'ll': 0.01,  'ul':0.1,  'variable type': float}}
const = {}

num_agents = [#{'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              #{'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              #{'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              #{'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              #{'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              #{'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              #{'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},
              {'mc':1, 'lhc': 1, 'h-c': 1, 'ns':1, 'ga':1, 'pymoo':1},]
bb = run_parallel_algorithm(dvs, objs, const, model, num_agents, base_time)

In [ ]:
def get_re_pf(benchmark, path_to_pf='projects/reproblems-master/approximated_Pareto_fronts/'):
    home = os.getenv("HOME")
    os.chdir(home)
    os.chdir(path_to_pf)
    pf = []
    with open('reference_points_{}.dat'.format(benchmark.upper()), 'r') as f:
        for line in f:
            pf.append([float(x) for x in line.split(' ')])
    return np.array(pf)
pf2 = get_re_pf('re32')


fig = go.Figure()
#fig.add_trace(go.Scatter(x=pf[:,0], y=pf[:,1], mode='markers', name='MABS'))
fig.add_trace(go.Scatter(x=pf2[:,0], y=pf2[:,1], mode='markers', name='True'))
fig.show()
fig = go.Figure()
#fig.add_trace(go.Scatter(x=pf[:,0], y=pf[:,2], mode='markers', name='MABS'))
fig.add_trace(go.Scatter(x=pf2[:,0], y=pf2[:,2], mode='markers', name='True'))
fig.show()
fig = go.Figure()
#fig.add_trace(go.Scatter(x=pf[:,1], y=pf[:,2], mode='markers', name='MABS'))
fig.add_trace(go.Scatter(x=pf2[:,1], y=pf2[:,2], mode='markers', name='True'))
fig.show()
fig = go.Figure()
#fig.add_trace(go.Scatter3d(x=pf[:,0], y=pf[:,1], z=pf[:,2], mode='markers', name='MABS'))
fig.add_trace(go.Scatter3d(x=pf2[:,0], y=pf2[:,1], z=pf2[:,2], mode='markers', name='True'))
fig.show()

# Welded Beam (RE32)

In [ ]:
model = 'welded_beam'
objs = {'f0': {'ll':0.0,       'ul':40.0, 'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,       'ul':0.015, 'goal':'lt', 'variable type': float},}
dvs =  {'x0':  {'ll': 0.125, 'ul':5.0,  'variable type': float},
        'x1':  {'ll': 0.1,   'ul':10.0, 'variable type': float},
        'x2':  {'ll': 0.1,   'ul':10.0, 'variable type': float},
        'x3':  {'ll': 0.125, 'ul':5.0,  'variable type': float}}
const = {'c0': {'ll': -1E6,   'ul':0.0,  'variable type': float},
         'c1': {'ll': -1E6,   'ul':0.0,  'variable type': float},
         'c2': {'ll': -1E6,   'ul':0.0,   'variable type': float},
         'c3': {'ll': -1E6,   'ul':0.0,   'variable type': float},}
dvi_ = {'f0': 200, 'f1':100}

problem = BenchmarkProblem(design_variables=dvs,
                         objectives=objs,
                         constraints=const,
                         benchmark_name = model) 

bb = {'name': 'bb_opt', 'type': bb_opt.BbOpt, 'attr': {'archive_name': '{}.h5'.format(model), 
                                                       'convergence_interval': 25,
                                                       'convergence_type': 'function evals',
                                                       'function_evals': 25_000,
                                                       'pf_size': 200,
                                                       'plot': False,
                                                       'meta_data_to_log': ['function evals',]}}#'hvi', 'gd', 'igd']}}

kas = {'ka_rp_explore': {'type': Stochastic, 'attr' : {}}, 
       'ka_rp_lhc': {'type': LatinHypercube, 'attr' : {'samples': len(dvs)*100,'debug_wait':False, 'execute_once': True}},
       'ka_rp_lhc2': {'type': LatinHypercube, 'attr' : {'samples': len(dvs)*100,'debug_wait':False, 'lhc_criterion': 'simple', 'execute_once': True}},
       'ka_rp_hc1': {'type': HillClimb, 'attr' : {'step_size':0.01,'step_limit':50, 'reanalyze_designs': True}},
       'ka_rp_hc2': {'type': HillClimb, 'attr' : {'hc_type':'steepest ascent','step_limit':100}},
       'ka_rp_pert1': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.01, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert4': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.01, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert5': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.01, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert6': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.01, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert7': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.01, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert8': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.01, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert9': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.11, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert10': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.09, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert11': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.08, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert12': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.07, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert13': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.06, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert14': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.05, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert2': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.005}},
       'ka_rp_pert3': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.005, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pymoo2': {'type': PyMooAlgorithm, 'attr' : {'pymoo_algorithm_name': 'nsga2', 'pop_size': 50, 'n_offspring': 50, 'termination_criteria':2500, 'execute_once': True}},
       'ka_rp_pymoo3': {'type': PyMooAlgorithm, 'attr' : {'pymoo_algorithm_name': 'nsga2', 'pop_size': 100, 'n_offspring': 100, 'termination_criteria':3000, 'execute_once': True}},
       'ka_rp_pymoo4': {'type': PyMooAlgorithm, 'attr' : {'pymoo_algorithm_name': 'nsga2', 'pop_size': 250, 'n_offspring': 250, 'termination_criteria':5000, 'execute_once': True}},
       'ka_br_lvl3': {'type': KaBrLevel3, 'attr' : {}},
       'ka_br_lvl2': {'type': KaBrLevel2, 'attr' : {}},
       'ka_br_lvl1': {'type': KaBrLevel1, 'attr' : {}}}#'total_pf_size':500, 'pareto_sorter':'dci', 'dci_div':dvi_}}}



for num in range(0,1):
    BenchmarkController(bb, kas, problem, num, rand=None)
    directory = 'welded_beam2'
    if not os.path.isdir('benchmarks/{}'.format(directory)):
        os.mkdir('benchmarks/{}'.format(directory))
    shutil.move('{}.h5'.format(model, num), 'benchmarks/{}/{}_{}.h5'.format(directory,model,num))
    time.sleep(5)    

# Truss2D

In [ ]:
model = 'truss2d'
objs = {'f0': {'ll':0.0,      'ul':0.055, 'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,      'ul':100_000, 'goal':'lt', 'variable type': float},}
dvs =  {'x0':  {'ll': 0.0,    'ul':0.01,  'variable type': float},
        'x1':  {'ll': 0.0,    'ul':0.01, 'variable type': float},
        'x2':  {'ll': 1.0,    'ul':3.0, 'variable type': float}}
const = {'c0': {'ll': -1E5,   'ul':0.0,  'variable type': float},}
dvi_ = {'f0': 100000, 'f1':55}

problem = BenchmarkProblem(design_variables=dvs,
                         objectives=objs,
                         constraints=const,
                         benchmark_name = model) 

bb = {'name': 'bb_opt', 'type': bb_opt.BbOpt, 'attr': {'archive_name': '{}.h5'.format(model), 
                                                       'convergence_interval': 10,
                                                       'convergence_type': 'function evals',
                                                       'function_evals': 10000,
                                                       'pf_size': 200,
                                                       'meta_data_to_log': ['hvi', 'gd', 'igd', 'function evals']}}

kas = {'ka_rp_explore': {'type': Stochastic, 'attr' : {}}, 
       'ka_rp_lhc': {'type': LatinHypercube, 'attr' : {'samples': len(dvs)*100,'debug_wait':False, 'execute_once': True}},
       'ka_rp_lhc2': {'type': LatinHypercube, 'attr' : {'samples': len(dvs)*100,'debug_wait':False, 'lhc_criterion': 'simple', 'execute_once': True}},
       'ka_rp_hc1': {'type': HillClimb, 'attr' : {'step_size':0.01,'step_limit':50, 'reanalyze_designs': True}},
       'ka_rp_hc2': {'type': HillClimb, 'attr' : {'hc_type':'steepest ascent','step_limit':100}},
       'ka_rp_pert1': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.01, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pert2': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.005}},
       'ka_rp_pert3': {'type': NeighborhoodSearch, 'attr' : {'perturbation_size':0.005, 'neighboorhood_search':'random','reanalyze_designs': True}},
       'ka_rp_pymoo2': {'type': PyMooAlgorithm, 'attr' : {'pymoo_algorithm_name': 'nsga2', 'pop_size': 50, 'n_offspring': 50, 'termination_criteria':2500, 'execute_once': True}},
       'ka_br_lvl3': {'type': KaBrLevel3, 'attr' : {}},
       'ka_br_lvl2': {'type': KaBrLevel2, 'attr' : {}},
       'ka_br_lvl1': {'type': KaBrLevel1, 'attr' : {}}}#'pareto_sorter':'dci', 'dci_div':dvi_}}}


for num in range(10,20):
    BenchmarkController(bb, kas, problem, num, rand=1)
    directory = 'truss2d'
    if not os.path.isdir('benchmarks/{}'.format(directory)):
        os.mkdir('benchmarks/{}'.format(directory))
    shutil.move('{}.h5'.format(model, num), 'benchmarks/{}/{}_{}.h5'.format(directory,model,num))
    time.sleep(5)  

 # RE21

In [ ]:
model = 're21'
objs = {'f0': {'ll':0.0,      'ul':3_000.0, 'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,      'ul':1.0, 'goal':'lt', 'variable type': float},}

a = float(np.sqrt(2.))

dvs =  {'x0':  {'ll': 1.0,  'ul':3.0,  'variable type': float},
        'x1':  {'ll': a,    'ul':3.0, 'variable type': float},
        'x2':  {'ll': a,    'ul':3.0, 'variable type': float},
        'x3':  {'ll': 1.0,  'ul':3.0, 'variable type': float},}
const = {}

run_algorithm(dvs, objs, const, model) 

# RE22

In [ ]:
model = 're22'
objs = {'f0': {'ll':0.0,    'ul':500.0, 'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,    'ul':50.0, 'goal':'lt', 'variable type': float},}
dvs =  {'x0':  {'options' : [0.20, 0.31, 0.40, 0.44, 0.60, 0.62, 0.79, 0.80, 0.88, 0.93, 1.0, 1.20, 
                             1.24, 1.32, 1.40, 1.55, 1.58, 1.60, 1.76, 1.80, 1.86, 2.0, 2.17, 2.20, 
                             2.37, 2.40, 2.48, 2.60, 2.64, 2.79, 2.80, 3.0, 3.08, 3,10, 3.16, 3.41, 
                             3.52, 3.60, 3.72, 3.95, 3.96, 4.0, 4.03, 4.20, 4.34, 4.40, 4.65, 4.74, 
                             4.80, 4.84, 5.0, 5.28, 5.40, 5.53, 5.72, 6.0, 6.16, 6.32, 6.60, 7.11, 
                             7.20, 7.80, 7.90, 8.0, 8.40, 8.69, 9.0, 9.48, 10.27, 11.0, 11.06, 11.85, 
                             12.0, 13.0, 14.0, 15.0], 'variable type': float},
        'x1':  {'ll': 0.0,  'ul':20.0, 'variable type': float},
        'x2':  {'ll': 0.0,  'ul':40.0, 'variable type': float},}
const = {}

run_algorithm(dvs, objs, const, model) 

# RE23

In [ ]:
model = 're23'
objs = {'f0': {'ll':0.0,    'ul':6_000.0, 'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,    'ul':1_300_000.0, 'goal':'lt', 'variable type': float},}
dvs =  {'x0':  {'options' : [x for x in range(100)], 'variable type': int},
        'x1':  {'options' : [x for x in range(100)], 'variable type': int},
        'x2':  {'ll': 10.0,  'ul':200.0, 'variable type': float},
        'x3':  {'ll': 10.0,  'ul':240.0, 'variable type': float},}
const = {}

run_algorithm(dvs, objs, const, model) 

# RE24

In [ ]:
model = 're24'

objs = {'f0': {'ll':0.0,      'ul':500.0, 'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,      'ul':50.0, 'goal':'lt', 'variable type': float},}
dvs =  {'x0':  {'ll': 0.5,  'ul':4.0,  'variable type': float},
        'x1':  {'ll': 0.5,  'ul':50.0, 'variable type': float},}
const = {}

run_algorithm(dvs, objs, const, model) 

# RE31

In [ ]:
model = 're31'

objs = {'f0': {'ll':0.0,      'ul':500,        'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,      'ul':8_500_000,  'goal':'lt', 'variable type': float},
        'f2': {'ll':0.0,      'ul':20_000_000, 'goal':'lt', 'variable type': float},}
dvs =  {'x0':  {'ll': 0.00001,  'ul':100.0,  'variable type': float},
        'x1':  {'ll': 0.00001,  'ul':100.0, 'variable type': float},
        'x2':  {'ll': 1.0,    'ul':3.0, 'variable type': float}}
const = {}

run_algorithm(dvs, objs, const, model) 

# RE32

In [ ]:
model = 're32'

objs = {'f0': {'ll':0.0,      'ul':35.0,        'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,      'ul':18_000,  'goal':'lt', 'variable type': float},
        'f2': {'ll':0.0,      'ul':426_000_000, 'goal':'lt', 'variable type': float},}
dvs =  {'x0':  {'ll': 0.125,  'ul':5.0,  'variable type': float},
        'x1':  {'ll': 0.1,    'ul':10.0, 'variable type': float},
        'x2':  {'ll': 0.1,    'ul':10.0, 'variable type': float},
        'x3':  {'ll': 0.125,  'ul':5.0, 'variable type': float}}
const = {}

run_algorithm(dvs, objs, const, model) 

# RE33

In [ ]:
model = 're33'

objs = {'f0': {'ll':-1.0,      'ul':6.0,   'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,      'ul':10.0,  'goal':'lt', 'variable type': float},
        'f2': {'ll':0.0,      'ul':4.5E9, 'goal':'lt', 'variable type': float},}
dvs =  {'x0':  {'ll': 55.0,   'ul':80.0,   'variable type': float},
        'x1':  {'ll': 75.0,   'ul':110.0,  'variable type': float},
        'x2':  {'ll': 1000.0, 'ul':3000.0, 'variable type': float},
        'x3':  {'ll': 11.0,   'ul':20.0,   'variable type': float}}
const = {}

run_algorithm(dvs, objs, const, model) 

# RE34

In [ ]:
model = 're34'

objs = {'f0': {'ll':1_650.0,  'ul':1_700.0, 'goal':'lt', 'variable type': float},
        'f1': {'ll':6.0,      'ul':11.0,    'goal':'lt', 'variable type': float},
        'f2': {'ll':0.0,      'ul':0.3,     'goal':'lt', 'variable type': float},}
dvs =  {'x0':  {'ll': 1.0,   'ul':3.0,  'variable type': float},
        'x1':  {'ll': 1.0,   'ul':3.0,  'variable type': float},
        'x2':  {'ll': 1.0,   'ul':3.0,  'variable type': float},
        'x3':  {'ll': 1.0,   'ul':3.0,  'variable type': float},
        'x4':  {'ll': 1.0,   'ul':3.0,  'variable type': float}}
const = {}

run_algorithm(dvs, objs, const, model) 

# RE35

In [ ]:
model = 're35'

objs = {'f0': {'ll':2_000.0,  'ul':7_100.0, 'goal':'lt', 'variable type': float},
        'f1': {'ll':650.0,      'ul':1_700.0,    'goal':'lt', 'variable type': float},
        'f2': {'ll':0.0,      'ul':400.0,     'goal':'lt', 'variable type': float},}
dvs =  {'x0':  {'ll': 2.6,   'ul':3.6,  'variable type': float},
        'x1':  {'ll': 0.7,   'ul':0.8,  'variable type': float},
        'x2':  {'options': [x for x in range(17,29)],  'variable type': int},
        'x3':  {'ll': 7.3,   'ul':8.3,  'variable type': float},
        'x4':  {'ll': 7.3,   'ul':8.3,  'variable type': float},
        'x5':  {'ll': 2.9,   'ul':3.9,  'variable type': float},
        'x6':  {'ll': 5.0,   'ul':5.5,  'variable type': float},}

const = {}

run_algorithm(dvs, objs, const, model) 

# RE36

In [ ]:
model = 're36'

objs = {'f0': {'ll':0.0,  'ul':6.0, 'goal':'lt', 'variable type': float},
        'f1': {'ll':11.0,      'ul':40.0,    'goal':'lt', 'variable type': float},
        'f2': {'ll':0.0,      'ul':0.36,     'goal':'lt', 'variable type': float},}
dvs =  {'x0':  {'options': [x for x in range(12,61)],  'variable type': int},
        'x1':  {'options': [x for x in range(12,61)],  'variable type': int},
        'x2':  {'options': [x for x in range(12,61)],  'variable type': int},
        'x3':  {'options': [x for x in range(12,61)],  'variable type': int},}

const = {}

run_algorithm(dvs, objs, const, model) 

# RE37

# RE41

In [ ]:
model = 're41'

objs = {'f0': {'ll':15.0, 'ul':45.0, 'goal':'lt', 'variable type': float},
        'f1': {'ll':3.5,  'ul':4.5, 'goal':'lt', 'variable type': float},
        'f2': {'ll':10.5, 'ul':13.1, 'goal':'lt', 'variable type': float},
        'f3': {'ll':0.0,  'ul':9.5, 'goal':'lt', 'variable type': float},       }
dvs =  {'x0':  {'ll': 0.5,   'ul':1.5,  'variable type': float},
        'x1':  {'ll': 0.45,  'ul':1.35,  'variable type': float},
        'x2':  {'ll': 0.5,   'ul':1.5,  'variable type': float},
        'x3':  {'ll': 0.5,   'ul':1.5,  'variable type': float},
        'x4':  {'ll': 0.875, 'ul':2.625,  'variable type': float},
        'x5':  {'ll': 0.4,   'ul':1.2,  'variable type': float},
        'x6':  {'ll': 0.4,   'ul':1.2,  'variable type': float},       }
const = {}


run_algorithm(dvs, objs, const, model) 

# RE61

In [ ]:
model = 're61'

objs = {'f0': {'ll':63_000.0, 'ul':77_000.0, 'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,  'ul':1_400.0, 'goal':'lt', 'variable type': float},   
        'f2': {'ll':250_000.0,  'ul':2_900_000.0, 'goal':'lt', 'variable type': float},
        'f3': {'ll':70_000.0, 'ul':16_000_000.0, 'goal':'lt', 'variable type': float},
        'f4': {'ll':0.0,  'ul':350_000.0, 'goal':'lt', 'variable type': float},
        'f5': {'ll':0.0,  'ul':95_000.0, 'goal':'lt', 'variable type': float}}
dvs =  {'x0':  {'ll': 0.01,  'ul':0.45,  'variable type': float},
        'x1':  {'ll': 0.01,  'ul':0.1,  'variable type': float},
        'x2':  {'ll': 0.01,  'ul':0.1,  'variable type': float}}
const = {}

run_algorithm(dvs, objs, const, model) 

# RE91

In [ ]:
model = 're91'

objs = {'f0': {'ll':15.0, 'ul':45.0, 'goal':'lt', 'variable type': float},
        'f1': {'ll':0.0,  'ul':1.4, 'goal':'lt', 'variable type': float}, 
        'f2': {'ll':0.0,  'ul':350.0, 'goal':'lt', 'variable type': float},        
        'f3': {'ll':0.0,  'ul':1.1, 'goal':'lt', 'variable type': float},
        'f4': {'ll':0.3,  'ul':1.6, 'goal':'lt', 'variable type': float},
        'f5': {'ll':0.5,  'ul':1.31, 'goal':'lt', 'variable type': float},
        'f6': {'ll':0.7,  'ul':1.3, 'goal':'lt', 'variable type': float},
        'f7': {'ll':0.6,  'ul':1.2, 'goal':'lt', 'variable type': float},
        'f8': {'ll':0.65, 'ul':1.05, 'goal':'lt', 'variable type': float},}

dvs =  {'x0':  {'ll': 0.5,   'ul':1.5,  'variable type': float},
        'x1':  {'ll': 0.45,  'ul':1.35,  'variable type': float},
        'x2':  {'ll': 0.5,   'ul':1.5,  'variable type': float},
        'x3':  {'ll': 0.5,   'ul':1.5,  'variable type': float},
        'x4':  {'ll': 0.875, 'ul':2.625,  'variable type': float},
        'x5':  {'ll': 0.4,   'ul':1.2,  'variable type': float},
        'x6':  {'ll': 0.4,   'ul':1.2,  'variable type': float},       }
const = {}

run_algorithm(dvs, objs, const, model)

In [ ]:
from src.utils.benchmark_utils import get_re_pf

pf2 = get_re_pf('re36')

for num in range(1,3):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pf2[:,0], y=pf2[:,num], mode='markers', name='True'))
    fig.show()